# Problem : QA Chatbot with interacting with Csv file is shown.

In [19]:
!pip install langchain openai chromadb tiktoken faiss-cpu
import openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import langchain
import chromadb
import tiktoken

In [21]:
from langchain.document_loaders import CSVLoader  #load csv files
from langchain.indexes import VectorstoreIndexCreator #importing VSindex
from langchain.chains import RetrievalQA #Q&A retrieval system
from langchain.llms import OpenAI #interact with the excel csv file

In [22]:
from langchain.embeddings  import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain import OpenAI, VectorDBQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [23]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "sk...7d"
openai.api_key = "sk...7d"

In [24]:
import pandas as pd
df = pd.read_csv("helpdesk-dataset.csv")

In [25]:
df_1 = df[df['Input'].str.contains("Solution can be found at https")== False]
df = df_1.drop(columns = "Source")

In [26]:
df = df_1.drop(columns = "Source")
df.head()


,Instruction,Input
0,I need to modify timesheet entries which are a...,Issue\n I need to modify timesheet entries th...
1,How to install Ivanti Secure Access Client fro...,"Dear Users, \nPlease note Pulse Secure has bee..."
4,Conditional Access for Atos Corporate Windows ...,This article presents the concept of the compl...
5,Microsoft Office 365 Services and Licensing,Purpose of this article\nTo understand basic M...
6,DAS - FAQ,Table of Contents\nDAS object types and Respon...


In [27]:
df.shape

(192, 2)

In [28]:
df["Instruction"] = df["Instruction"] + df["Input"]
df_1 = df.drop(columns = 'Input')

In [29]:
df_1.head()

,Instruction
0,I need to modify timesheet entries which are a...
1,How to install Ivanti Secure Access Client fro...
4,Conditional Access for Atos Corporate Windows ...
5,Microsoft Office 365 Services and LicensingPur...
6,DAS - FAQTable of Contents\nDAS object types a...


In [30]:
df_1.to_csv("df_1.txt", index = False, header = False)


# Text splitter, Embeddings, Vector Store

In [31]:
with open("df_1.txt") as f:
  df_1 = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_text(df_1)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(texts, embeddings)


#QA with vectorDB. 

In [35]:
qa = VectorDBQA.from_chain_type(llm = OpenAI(), chain_type = "stuff", vectorstore = vectorstore)


In [36]:
query_ = "What is timesheet modification?"

In [37]:
qa.run(query_)

' Timesheet modification is when an end user needs to make changes to timesheet entries that are already released or approved. This is usually done by contacting the respective Project Manager to request the changes via the O2C portal, or by the local time administrator.'

In [38]:
query_ = "How to not have redundant text blocks in Outlook?"
qa.run(query_)

' To remove a reusable text block, follow the below steps:\n  1. Open New Email and click on Insert tab\n2. Go to Quick Parts - right-click on the text message and click on Organize and Delete\n  3. Select the message to delete and click on Delete.\n  Source: https://support.microsoft.com/en-us/office/create-reuseable-text-blocks-for-email-messages-8fb6c723-c960-4c8c-9790-3e43ddc4b186'